In [13]:
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

# Definir la ruta de la carpeta con las imágenes
path = '/content/drive/MyDrive/chest_xray'
train_path = '/content/drive/MyDrive/chest_xray/train'
test_path = '/content/drive/MyDrive/chest_xray/test'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Cargar el modelo pre-entrenado de ResNet50
base_model = ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


In [ ]:
# Añadir una capa de pooling global promedio en la parte superior del modelo base
x = base_model.output
x = GlobalAveragePooling2D()(x)

In [ ]:
# Añadir una capa densa de salida con una neurona (para clasificación binaria)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
# Crear el modelo final para entrenar
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
# Congelar las capas del modelo base para el transfer learning
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Compilar el modelo
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Crear generadores de imágenes (puedes cambiar los directorios a tus rutas de datos)
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 5218 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [15]:
# Calcular los pasos por época
steps_per_epoch = len(train_generator)
validation_steps = len(validation_generator)

In [17]:
# Entrenar el modelo
model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_steps)

 87/164 [==============>...............] - ETA: 6:35 - loss: 0.5693 - accuracy: 0.7418

KeyboardInterrupt: 

In [ ]:
# Crear un generador de imágenes para tus datos de prueba
test_generator = test_datagen.flow_from_directory(
    'raw/test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False)  # Importante: no mezclar los datos de prueba

In [ ]:
# Usar el modelo para hacer predicciones en los datos de prueba
predictions = model.predict(test_generator, steps=len(test_generator), verbose=1)

In [ ]:
# Las predicciones serán probabilidades de pertenecer a la clase 1
# Podemos convertirlas a etiquetas de clase binarias estableciendo un umbral
threshold = 0.5
predictions = [1 if p > threshold else 0 for p in predictions]

# Ahora puedes comparar estas predicciones con las etiquetas reales
accuracy = accuracy_score(test_generator.classes, predictions)

print("La precisión del modelo en el conjunto de prueba es: ", accuracy)